In [ ]:
- Author: Ben Du
- Date: 2020-06-17 15:24:04
- Title: Types of Joins of Spark DataFrames
- Slug: spark-dataframe-types-joins
- Category: Computer Science
- Tags: Computer Science, Spark, DataFrame, join, type, inner join, outer join, left join, right join, full join, big data

http://kirillpavlov.com/blog/2016/04/23/beyond-traditional-join-with-apache-spark/

## Comments

1. it is suggested that you always use `Seq(c1, c2, ...)` even if there's only one field for joining. 

In [1]:
%%classpath add mvn
org.apache.spark spark-core_2.11 2.3.1
org.apache.spark spark-sql_2.11 2.3.1
org.apache.spark spark-hive_2.11 2.3.1

In [2]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
    .master("local[2]")
    .appName("Types of Joins in Spark")
    .getOrCreate()
spark

org.apache.spark.sql.SparkSession@4a82094b

In [3]:
import spark.implicits._

org.apache.spark.sql.SparkSession$implicits$@7e391d71

In [4]:
val employees = Seq(
  ("Rafferty", Some(31)), 
  ("Jones", Some(33)), 
  ("Heisenberg", Some(33)), 
  ("Robinson", Some(34)), 
  ("Smith", Some(34)), 
  ("Ben", Some(50)),
  ("Williams", null)
).toDF("LastName", "DepartmentID")

employees.show()

+----------+------------+
|  LastName|DepartmentID|
+----------+------------+
|  Rafferty|          31|
|     Jones|          33|
|Heisenberg|          33|
|  Robinson|          34|
|     Smith|          34|
|       Ben|          50|
|  Williams|        null|
+----------+------------+



null

In [5]:
val departments = Seq(
  (31, "Sales"), 
  (33, "Engineering"), 
  (34, "Clerical"),
  (35, "Marketing")
).toDF("DepartmentID", "DepartmentName")
departments.show()

+------------+--------------+
|DepartmentID|DepartmentName|
+------------+--------------+
|          31|         Sales|
|          33|   Engineering|
|          34|      Clerical|
|          35|     Marketing|
+------------+--------------+



null

## Inner Join

In [6]:
employees.join(departments, Seq("DepartmentID")).show()

+------------+----------+--------------+
|DepartmentID|  LastName|DepartmentName|
+------------+----------+--------------+
|          31|  Rafferty|         Sales|
|          33|     Jones|   Engineering|
|          33|Heisenberg|   Engineering|
|          34|  Robinson|      Clerical|
|          34|     Smith|      Clerical|
+------------+----------+--------------+



## Left Outer Join

1. For outer join, the joining keys must be passed in as a Seq of strings.

2. By default the joining keys from the left table are kept. 
you can still refer to keys in the right table by specifying the table name.

In [7]:
employees.join(departments, Seq("DepartmentID"), "left_outer").show()

+------------+----------+--------------+
|DepartmentID|  LastName|DepartmentName|
+------------+----------+--------------+
|          31|  Rafferty|         Sales|
|          33|     Jones|   Engineering|
|          33|Heisenberg|   Engineering|
|          34|  Robinson|      Clerical|
|          34|     Smith|      Clerical|
|          50|       Ben|          null|
|        null|  Williams|          null|
+------------+----------+--------------+



In [5]:
employees.join(departments, Seq("DepartmentID"), "left_outer").
    filter(departments("DepartmentID").isNull).
    show()

+------------+--------+--------------+
|DepartmentID|LastName|DepartmentName|
+------------+--------+--------------+
|          50|     Ben|          null|
|        null|Williams|          null|
+------------+--------+--------------+



In [11]:
employees.join(departments, Seq("DepartmentID"), "left_outer").
    withColumn("has_depart", departments("DepartmentID").isNotNull).
    show()

Name: org.apache.spark.sql.AnalysisException
Message: Resolved attribute(s) DepartmentID#21 missing from DepartmentID#6,LastName#5,DepartmentName#22 in operator !Project [DepartmentID#6, LastName#5, DepartmentName#22, isnotnull(DepartmentID#21) AS has_depart#95]. Attribute(s) with the same name appear in the operation: DepartmentID. Please check if the right attribute(s) are used.;;
!Project [DepartmentID#6, LastName#5, DepartmentName#22, isnotnull(DepartmentID#21) AS has_depart#95]
+- Project [DepartmentID#6, LastName#5, DepartmentName#22]
   +- Join LeftOuter, (DepartmentID#6 = DepartmentID#21)
      :- Project [_1#2 AS LastName#5, _2#3 AS DepartmentID#6]
      :  +- LocalRelation [_1#2, _2#3]
      +- Project [_1#18 AS DepartmentID#21, _2#19 AS DepartmentName#22]
         +- LocalRelation [_1#18, _2#19]

StackTrace: !Project [DepartmentID#6, LastName#5, DepartmentName#22, isnotnull(DepartmentID#21) AS has_depart#95]
+- Project [DepartmentID#6, LastName#5, DepartmentName#22]
   +- Jo

In [12]:
employees.join(departments, Seq("DepartmentID"), "left_outer").
    select(departments("DepartmentID").isNotNull.alias("has_depart")).
    show()

lastException = null


Name: org.apache.spark.sql.AnalysisException
Message: Resolved attribute(s) DepartmentID#21 missing from DepartmentID#6,LastName#5,DepartmentName#22 in operator !Project [isnotnull(DepartmentID#21) AS has_depart#99]. Attribute(s) with the same name appear in the operation: DepartmentID. Please check if the right attribute(s) are used.;;
!Project [isnotnull(DepartmentID#21) AS has_depart#99]
+- Project [DepartmentID#6, LastName#5, DepartmentName#22]
   +- Join LeftOuter, (DepartmentID#6 = DepartmentID#21)
      :- Project [_1#2 AS LastName#5, _2#3 AS DepartmentID#6]
      :  +- LocalRelation [_1#2, _2#3]
      +- Project [_1#18 AS DepartmentID#21, _2#19 AS DepartmentName#22]
         +- LocalRelation [_1#18, _2#19]

StackTrace: !Project [isnotnull(DepartmentID#21) AS has_depart#99]
+- Project [DepartmentID#6, LastName#5, DepartmentName#22]
   +- Join LeftOuter, (DepartmentID#6 = DepartmentID#21)
      :- Project [_1#2 AS LastName#5, _2#3 AS DepartmentID#6]
      :  +- LocalRelation [_1#

In [14]:
employees.join(departments, employees("DepartmentID") === departments("DepartmentID"), "left_outer")
    .withColumn("has_depart", departments("DepartmentID").isNotNull)
    .show

+----------+------------+------------+--------------+----------+
|  LastName|DepartmentID|DepartmentID|DepartmentName|has_depart|
+----------+------------+------------+--------------+----------+
|  Rafferty|          31|          31|         Sales|      true|
|     Jones|          33|          33|   Engineering|      true|
|Heisenberg|          33|          33|   Engineering|      true|
|  Robinson|          34|          34|      Clerical|      true|
|     Smith|          34|          34|      Clerical|      true|
|       Ben|          50|        null|          null|     false|
|  Williams|        null|        null|          null|     false|
+----------+------------+------------+--------------+----------+



In [15]:
val depart2 = Seq(
  (31, "Sales"), 
  (33, "Engineering"), 
  (34, "Clerical"),
  (35, "Marketing")
).toDF("depart_id", "depart_name")
depart2.show()

+---------+-----------+
|depart_id|depart_name|
+---------+-----------+
|       31|      Sales|
|       33|Engineering|
|       34|   Clerical|
|       35|  Marketing|
+---------+-----------+



depart2 = [depart_id: int, depart_name: string]


[depart_id: int, depart_name: string]

In [18]:
employees.join(depart2, $"DepartmentID" === $"depart_id", "left_outer")
    .withColumn("has_depart", $"depart_id".isNotNull)
    .show

+----------+------------+---------+-----------+----------+
|  LastName|DepartmentID|depart_id|depart_name|has_depart|
+----------+------------+---------+-----------+----------+
|  Rafferty|          31|       31|      Sales|      true|
|     Jones|          33|       33|Engineering|      true|
|Heisenberg|          33|       33|Engineering|      true|
|  Robinson|          34|       34|   Clerical|      true|
|     Smith|          34|       34|   Clerical|      true|
|       Ben|          50|     null|       null|     false|
|  Williams|        null|     null|       null|     false|
+----------+------------+---------+-----------+----------+



Spark allows using following join types: 
inner, outer, left_outer, right_outer, leftsemi. 
The interface is the same as for left outer join in the example above.

## Right Outer Join

In [33]:
employees.join(departments, Seq("DepartmentID"), "right_outer").show()

+------------+----------+--------------+
|DepartmentID|  LastName|DepartmentName|
+------------+----------+--------------+
|          31|  Rafferty|         Sales|
|          33|Heisenberg|   Engineering|
|          33|     Jones|   Engineering|
|          34|     Smith|      Clerical|
|          34|  Robinson|      Clerical|
|          35|      null|     Marketing|
+------------+----------+--------------+



## A - B

In [50]:
employees.join(departments, employees("DepartmentID") === departments("DepartmentID"), "left_outer").show

+----------+------------+------------+--------------+
|  LastName|DepartmentID|DepartmentID|DepartmentName|
+----------+------------+------------+--------------+
|  Rafferty|          31|          31|         Sales|
|     Jones|          33|          33|   Engineering|
|Heisenberg|          33|          33|   Engineering|
|  Robinson|          34|          34|      Clerical|
|     Smith|          34|          34|      Clerical|
|       Ben|          50|        null|          null|
|  Williams|        null|        null|          null|
+----------+------------+------------+--------------+



In [53]:
employees.join(departments, Seq("DepartmentID"), "left_outer").show

+------------+----------+--------------+
|DepartmentID|  LastName|DepartmentName|
+------------+----------+--------------+
|          31|  Rafferty|         Sales|
|          33|     Jones|   Engineering|
|          33|Heisenberg|   Engineering|
|          34|  Robinson|      Clerical|
|          34|     Smith|      Clerical|
|          50|       Ben|          null|
|        null|  Williams|          null|
+------------+----------+--------------+



In [43]:
employees.join(departments, employees("DepartmentID") === departments("DepartmentID"), "left_outer").
    filter(departments("DepartmentID").isNull).show

+--------+------------+------------+--------------+
|LastName|DepartmentID|DepartmentID|DepartmentName|
+--------+------------+------------+--------------+
|     Ben|          50|        null|          null|
|Williams|        null|        null|          null|
+--------+------------+------------+--------------+



In [49]:
employees.join(departments, Seq("DepartmentID"), "left_outer").
    filter(departments("DepartmentID").isNull).show

+------------+--------+--------------+
|DepartmentID|LastName|DepartmentName|
+------------+--------+--------------+
|          50|     Ben|          null|
|        null|Williams|          null|
+------------+--------+--------------+



In [47]:
employees.join(departments, Seq("DepartmentID"), "left_outer").
    filter(employees("DepartmentID").isNull).show

+------------+--------+--------------+
|DepartmentID|LastName|DepartmentName|
+------------+--------+--------------+
|        null|Williams|          null|
+------------+--------+--------------+



In [48]:
employees.join(departments, Seq("DepartmentID"), "left_outer").
    filter($"DepartmentID".isNull).show

+------------+--------+--------------+
|DepartmentID|LastName|DepartmentName|
+------------+--------+--------------+
|        null|Williams|          null|
+------------+--------+--------------+



## Cartesian Join

Notice that you have to have "spark.sql.crossJoin.enabled" set to `true` 
in order to perform cartesian join on 2 DataFrames.

In [15]:
import org.apache.spark.sql.SparkSession

val sparkSession = SparkSession.
    builder().
    appName("Spark SQL basic example").
    config("spark.sql.crossJoin.enabled", "true").
    getOrCreate()
import sparkSession.implicits._

In [16]:
employees.join(departments).show

+----------+------------+------------+--------------+
|  LastName|DepartmentID|DepartmentID|DepartmentName|
+----------+------------+------------+--------------+
|  Rafferty|          31|          31|         Sales|
|  Rafferty|          31|          33|   Engineering|
|  Rafferty|          31|          34|      Clerical|
|  Rafferty|          31|          35|     Marketing|
|     Jones|          33|          31|         Sales|
|     Jones|          33|          33|   Engineering|
|     Jones|          33|          34|      Clerical|
|     Jones|          33|          35|     Marketing|
|Heisenberg|          33|          31|         Sales|
|Heisenberg|          33|          33|   Engineering|
|Heisenberg|          33|          34|      Clerical|
|Heisenberg|          33|          35|     Marketing|
|  Robinson|          34|          31|         Sales|
|  Robinson|          34|          33|   Engineering|
|  Robinson|          34|          34|      Clerical|
|  Robinson|          34|   

In [21]:
val products = Seq(
  ("steak", "1990-01-01", "2000-01-01", 150),
  ("steak", "2000-01-02", "2020-01-01", 180),
  ("fish", "1990-01-01", "2020-01-01", 100)
).toDF("name", "startDate", "endDate", "price")

products.show

+-----+----------+----------+-----+
| name| startDate|   endDate|price|
+-----+----------+----------+-----+
|steak|1990-01-01|2000-01-01|  150|
|steak|2000-01-02|2020-01-01|  180|
| fish|1990-01-01|2020-01-01|  100|
+-----+----------+----------+-----+



In [23]:
val orders = Seq(
  ("1995-01-01", "steak"),
  ("2000-01-01", "fish"),
  ("2005-01-01", "steak")
).toDF("date", "product")

orders.show

+----------+-------+
|      date|product|
+----------+-------+
|1995-01-01|  steak|
|2000-01-01|   fish|
|2005-01-01|  steak|
+----------+-------+



In [25]:
orders.
    join(products, $"product" === $"name" && $"date" >= $"startDate" && $"date" <= $"endDate").
    show()

+----------+-------+-----+----------+----------+-----+
|      date|product| name| startDate|   endDate|price|
+----------+-------+-----+----------+----------+-----+
|1995-01-01|  steak|steak|1990-01-01|2000-01-01|  150|
|2000-01-01|   fish| fish|1990-01-01|2020-01-01|  100|
|2005-01-01|  steak|steak|2000-01-02|2020-01-01|  180|
+----------+-------+-----+----------+----------+-----+

